In [1]:
import jsonlines
import random

In [2]:
def read_jsonl(file_paths):
    content_id = 0
    preprocessed_data = []
    data = []
    for file_path in file_paths:
        with open(file_path, "r") as f:
            for item in jsonlines.Reader(f):
                for section in item["sections"]:
                    start = section[0]
                    end = section[1]
                    content = " ".join(item["words"][start:end])
                    preprocessed_data.append({"id": "scirex-{:0>4d}".format(content_id), "content": content})
                    content_id += 1
                for relation in item["n_ary_relations"]:
                    Material = relation["Material"]
                    Method = relation["Method"]
                    Metric = relation["Metric"]
                    Task = relation["Task"]
                    score = relation["score"]
                    data.append({"Material": Material, "Method": Method, "Metric": Metric, "Task": Task, "score": score})
    return data

In [3]:
file_paths = ["/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/train.jsonl", 
              "/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/dev.jsonl", 
              "/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/test.jsonl"]
data = read_jsonl(file_paths)

In [4]:
import pandas as pd
for i in range(len(data)):
    if type(data[i]["score"]) == str:
        # print(data[i])
        if data[i]["score"] == "---":
            data[i]["score"] = "-"
        if "\ufeff" in data[i]["score"]:
            data[i]["score"] = data[i]["score"].replace("\ufeff", "")
        if "," in data[i]["score"]:
            data[i]["score"] = data[i]["score"].replace(",", ".")
        if not "±" in data[i]["score"] and not "/" in data[i]["score"] and not data[i]["score"] == "-" and not "%" in data[i]["score"] and not "m" in data[i]["score"] and not "M" in data[i]["score"] and not "k" in data[i]["score"] and not "K" in data[i]["score"] and not data[i]["score"] == "" and not "B" in data[i]["score"]:
            data[i]["score"] = float(data[i]["score"])
df = pd.DataFrame(data)
print(len(df))

2148


In [5]:
print(df.iloc[:50]["score"], type(df.iloc[:50]["score"]))

0         2.1
1       71.8%
2       469.0
3       71.8%
4       0.869
5        76.9
6        79.6
7       10.64
8       23.96
9       33.45
10      15.69
11      15.68
12      36.74
13       25.9
14       28.5
15     73.744
16     67.974
17     81.525
18     77.323
19       85.1
20       86.7
21       190k
22     5185.0
23     4650.0
24    5.2667%
25       0.95
26       0.88
27      0.778
28      0.686
29      0.881
30      0.956
31      0.819
32      0.627
33      0.498
34      80.9%
35      95.6%
36      71.18
37      69.21
38      61.41
39      69.73
40      65.02
41      57.19
42       63.1
43       58.2
44      68.07
45      68.16
46       8.59
47       55.8
48       27.0
49      507.0
Name: score, dtype: object <class 'pandas.core.series.Series'>


In [6]:
questions = []
question_id = 0
answers1 = []
answers2 = []
answers3 = []
answers4 = []
while question_id < 100:
    random_indices = random.randint(0, len(df)-1)
    row = df.iloc[random_indices]
    Material = row["Material"]
    Method = row["Method"]
    Metric = row["Metric"]
    Task = row["Task"]
    score = row["score"]
    subdata_no_Material = df.loc[(df["Method"] == Method) & (df["Metric"] == Metric) & (df["Task"] == Task)]
    subdata_no_Method = df.loc[(df["Material"] == Material) & (df["Metric"] == Metric) & (df["Task"] == Task)]
    subdata_no_Metric = df.loc[(df["Material"] == Material) & (df["Method"] == Method) & (df["Task"] == Task)]
    subdata_no_Task = df.loc[(df["Material"] == Material) & (df["Method"] == Method) & (df["Metric"] == Metric)]
    question_indices = random.randint(0, 3)
    if question_indices == 0 and len(subdata_no_Material) != 1:
        question = "On which dataset does the {} method achieve the highest {} score for {} task?".format(Method, Metric, Task)
        # print(subdata_no_Material)
        subdata_no_Material = subdata_no_Material.sort_values(by=["score"], ascending=False)
        answer = subdata_no_Material.iloc[0]["Material"]
        # answers1.append(answer)
    elif question_indices == 1 and len(subdata_no_Method) != 1:
        question = "Which method achieves the highest {} score on {} dataset for {} task?".format(Metric, Material, Task)
        subdata_no_Method = subdata_no_Method.sort_values(by=["score"], ascending=False)
        answer = subdata_no_Method.iloc[0]["Method"]
        # answers2.append(answr)
    elif question_indices == 2:
        question = "On what metrics is the {} method evaluated on {} dataset for {} task?".format(Method, Material, Task)
        metrics = subdata_no_Metric["Metric"].unique()
        answer = ", ".join(metrics)
        # answers3.append(answer)
    elif question_indices == 3:
        question = "Which datasets is {} method evaluated on for {} task?".format(Method, Task)
        materials = subdata_no_Task["Material"].unique()
        answer = ", ".join(materials)
    else:
        continue
        # answers4.append(answer)
    if (question_indices == 2 and answer not in answers3):
        answers3.append(answer)
        questions.append({"qid": "hard-scirex-{:0>4d}".format(question_id), "question": question, "answer": answer})
        question_id += 1
    elif (question_indices == 3 and answer not in answers4):
        answers4.append(answer)
        questions.append({"qid": "hard-scirex-{:0>4d}".format(question_id), "question": question, "answer": answer})
        question_id += 1
    elif (question_indices == 0 and answer not in answers1):
        answers1.append(answer)
        questions.append({"qid": "hard-scirex-{:0>4d}".format(question_id), "question": question, "answer": answer})
        question_id += 1
    elif (question_indices == 1 and answer not in answers2):
        answers2.append(answer)
        questions.append({"qid": "hard-scirex-{:0>4d}".format(question_id), "question": question, "answer": answer})
        question_id += 1
    else:
        continue


In [7]:
with jsonlines.open('/<YOUR_OWN_PATH>/ToolQA/data/questions/hard/scirex-hard.jsonl', mode='w') as writer:
    for row in questions:
        writer.write(row)